In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
def hamming(arr1, arr2):
    return np.sum(arr1 != arr2)/arr1.shape[0]

In [3]:
def avg_hamming(curr, other):
    return np.mean((curr != other).sum(axis = 0)/other.shape[1])

In [4]:
def greedy(s):
    np.random.seed(1)

    idx = np.random.choice(np.arange(s.shape[0]))

    chosen = s[[idx]]

    s = np.delete(s, idx, 0)

    for i in range(s.shape[0]):
        hammings = np.apply_along_axis(lambda x: avg_hamming(x, chosen), 1, s)

        chosen = np.vstack([chosen, s[hammings.argmax()]])

        s = np.delete(s, hammings.argmax(), 0)

    return chosen

In [5]:
for instance in os.listdir('../../mps_files'):
    name = instance.split('.')[0]
    output_path = f'hamming_results/{name}.txt'

    if os.path.exists(output_path):
        os.remove(output_path)
    
    with open(output_path, 'w') as f:
        f.write('sol_number,avg_hamming,weights,Method\n')
        f.close()

    for results in os.listdir('unique_results'):
        if name in results:
            if 'random' in results:
                weights_type = 'random'
            else:
                weights_type = 'uniform'

            sols = np.array(pd.read_csv(f'unique_results/{results}', header = None))[:100]

            shuffled = sols.copy()
            np.random.shuffle(shuffled)

            greedy_sols = greedy(sols)

            solution_sets = [sols, shuffled, greedy_sols]
            methods = ['Solution Pool', 'Random Shuffle', 'Greedy']

            for method, solutions in zip(methods, solution_sets):
                hamming_averages = []
                hamming_distances = []
                for i in range(1, 100):
                    for j in range(i):
                        hamming_distances.append(hamming(solutions[i], solutions[j]))
                    hamming_averages.append(f'{i},{np.array(hamming_distances).mean()},{weights_type},{method}')

                with open(output_path, 'a') as f:
                    f.write('\n'.join(hamming_averages) + '\n')
                    f.close()